In [1]:
import efficientnet.tfkeras as efn  # Convolutional Neural Network architecture
import IPython.display as ipd
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input


2022-11-19 18:47:47.168226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Convolutional Neural Network


In [2]:
IM_SIZE = (224, 224)
BIRDS = [
    "0se",
    "1hoang-yen",
    "2hoa-mi",
    "3tri",
    "4trich-co",
    "5hoet-lua",
    "6le-le",
    "7sao-nau",
    "9chich-choe",
    "8cu-dat",
    "10cong-coc",
    "11ca-cuong",
    "12co",
    "13quoc",
    "14hec-xoan",
    "15hoet-den",
    "16phuong-hoang-dat",
    "17oanh",
    "18huyt-co",
    "19khuou",
    "20trich-re",
    "21chang-nghich",
    "22mo-nhat",
    "23de-giun",
    "24khuyen",
    "25bim-bip",
    "26chao-mao",
    "27hut-mat",
    "28cheo-beo",
    "29cut",
    "30sau",
    "31cum-num",
    "32thanh-lam",
    "33bo-cau",
    "34choc-quach",
    "35que-lam",
    "36ngu-sac",
    "37hoanh-hoach",
    "38en",
    "39thanh-tuoc",
    "40vac",
    "41chia-voi",
    "42mat-xeo",
    "43mat-do",
    "44sam-cam",
    "45cu-gay",
    "46ket",
    "47trao-trao",
    "48xanh-tim",
]
DATA_PATH = "/Volumes/MacDATA/LEARNING/thesis/bird-recognition/core-ai/data/full-data/"
BATCH_SIZE = 16
MODEL_PATH = "/Volumes/MacDATA/LEARNING/thesis/bird-recognition/core-ai/models/mobilenet.h5"


In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    fill_mode="nearest",
)
train_batches = train_datagen.flow_from_directory(
    DATA_PATH + "train",
    classes=BIRDS,
    target_size=IM_SIZE,
    class_mode="categorical",
    shuffle=True,
    batch_size=BATCH_SIZE,
)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory(
    DATA_PATH + "val",
    classes=BIRDS,
    target_size=IM_SIZE,
    class_mode="categorical",
    shuffle=False,
    batch_size=BATCH_SIZE,
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batches = test_datagen.flow_from_directory(
    DATA_PATH + "test",
    classes=BIRDS,
    target_size=IM_SIZE,
    class_mode="categorical",
    shuffle=False,
    batch_size=BATCH_SIZE,
)

Found 12390 images belonging to 49 classes.
Found 1530 images belonging to 49 classes.
Found 1600 images belonging to 49 classes.


In [4]:
# Define CNN's architecture
net = MobileNetV2(include_top=False,
                  weights='imagenet',
                  input_tensor=None,
                  input_shape=(224, 224, 3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(BIRDS), activation="softmax", name="softmax")(x)
net_final = Model(inputs=net.input, outputs=output_layer)
net_final.compile(
    optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"]
)

# print(net_final.summary())


2022-11-19 18:47:52.137025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Estimate class weights for unbalanced dataset
class_weights = class_weight.compute_class_weight(
    "balanced", classes=np.unique(train_batches.classes), y=train_batches.classes
)

class_weights = {i:w for i,w in enumerate(class_weights)}

# Define callbacks
ModelCheck = ModelCheckpoint(
    MODEL_PATH,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

ReduceLR = ReduceLROnPlateau(
    monitor="val_loss", factor=0.2, patience=5, min_lr=3e-4)


In [6]:
# Train the model
net_final.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=30,
    class_weight=class_weights,
    callbacks=[ModelCheck, ReduceLR],
)

Epoch 1/30
775/775 [==============================] - 1002s 1s/step - loss: 5.8501 - accuracy: 0.0643 - val_loss: 55.8847 - val_accuracy: 0.0536 - lr: 0.0010
Epoch 2/30
775/775 [==============================] - 914s 1s/step - loss: 4.8397 - accuracy: 0.0413 - val_loss: 104.3001 - val_accuracy: 0.0634 - lr: 0.0010
Epoch 3/30
775/775 [==============================] - 912s 1s/step - loss: 4.3027 - accuracy: 0.0447 - val_loss: 44.0636 - val_accuracy: 0.0824 - lr: 0.0010
Epoch 4/30
775/775 [==============================] - 919s 1s/step - loss: 3.9830 - accuracy: 0.0834 - val_loss: 87.1635 - val_accuracy: 0.0451 - lr: 0.0010
Epoch 5/30
775/775 [==============================] - 919s 1s/step - loss: 3.4718 - accuracy: 0.1099 - val_loss: 19.2196 - val_accuracy: 0.1046 - lr: 0.0010
Epoch 6/30
775/775 [==============================] - 934s 1s/step - loss: 3.0754 - accuracy: 0.1472 - val_loss: 17.1719 - val_accuracy: 0.1222 - lr: 0.0010
Epoch 7/30
775/775 [==============================] - 94

Evaluate model

In [7]:
# show class indices
print('****************')
for cls, idx in test_batches.class_indices.items():
    print('Class nr ',idx,' -> ', cls)
print('****************')

loaded_model = Model(inputs=net.input, outputs=output_layer)

loaded_model.load_weights(MODEL_PATH)

# evaluate loaded model on test data
loaded_model.compile(optimizer=Adam(learning_rate=5e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
                  
score = loaded_model.evaluate(test_batches, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Y_pred = loaded_model.predict(test_batches)

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_batches.classes, y_pred))
print('Classification Report')
print(classification_report(test_batches.classes, y_pred, target_names=BIRDS))

****************
Class nr  0  ->  0se
Class nr  1  ->  1hoang-yen
Class nr  2  ->  2hoa-mi
Class nr  3  ->  3tri
Class nr  4  ->  4trich-co
Class nr  5  ->  5hoet-lua
Class nr  6  ->  6le-le
Class nr  7  ->  7sao-nau
Class nr  8  ->  9chich-choe
Class nr  9  ->  8cu-dat
Class nr  10  ->  10cong-coc
Class nr  11  ->  11ca-cuong
Class nr  12  ->  12co
Class nr  13  ->  13quoc
Class nr  14  ->  14hec-xoan
Class nr  15  ->  15hoet-den
Class nr  16  ->  16phuong-hoang-dat
Class nr  17  ->  17oanh
Class nr  18  ->  18huyt-co
Class nr  19  ->  19khuou
Class nr  20  ->  20trich-re
Class nr  21  ->  21chang-nghich
Class nr  22  ->  22mo-nhat
Class nr  23  ->  23de-giun
Class nr  24  ->  24khuyen
Class nr  25  ->  25bim-bip
Class nr  26  ->  26chao-mao
Class nr  27  ->  27hut-mat
Class nr  28  ->  28cheo-beo
Class nr  29  ->  29cut
Class nr  30  ->  30sau
Class nr  31  ->  31cum-num
Class nr  32  ->  32thanh-lam
Class nr  33  ->  33bo-cau
Class nr  34  ->  34choc-quach
Class nr  35  ->  35que-la